In [ ]:
# 将数据载入到 DataLoader 中
batch_size = 1000
train_data = FactorData(train_x.values, train_y.values)
# train_data = FactorData(train_x_converted, train_y_converted)
train_loader = DataLoader(dataset=train_data,
                          batch_size=batch_size,
                          shuffle=False)  # 不打乱数据集
test_data = FactorData(test_x.values, test_y.values)
# test_data = FactorData(test_x_converted, test_y_converted)
test_loader = DataLoader(dataset=test_data,
                         batch_size=batch_size,
                         shuffle=False)  # 不打乱数据集

# 构建模型
# alphanet = AlphaNet(combination=combination, combination_rev=combination_rev,
#                     index_list=index_list, fc1_num=8820, fc2_num=55, fc3_num=0, dropout_rate=0.2)
mlp = MLP(fc1_num=120, fc2_num=60, fc3_num=0, dropout_rate=0.2)
weight_list, bias_list = [], []
for name, p in mlp.named_parameters():
    # 将所有的 bias 参数放入 bias_list 中
    if 'bias' in name:
        bias_list += [p]
    # 将所有的 weight 参数放入 weight_list 中
    else:
        weight_list += [p]

# weight decay: 对所有 weight 参数进行 L2 正则化
optimizer = optim.RMSprop([{'params': weight_list, 'weight_decay': 1e-5},
                           {'params': bias_list, 'weight_decay': 0}],
                          lr=1e-4,
                          momentum=0.9)

# 损失函数为均方误差 MSE
criterion = nn.MSELoss()
test_criterion = nn.L1Loss()
epoch_num = 50
train_loss_list = []
test_loss_list = []
best_test_epoch, best_test_loss = 0, np.inf
seed = 0
for epoch in range(1, epoch_num+1):
    # 测试模式
    test_loss = 0
    mlp.eval()
    test_batch_num = 0
    with torch.no_grad():
        for data, label in tqdm(test_loader, f'Epoch {epoch}-test ', leave=False):
            test_batch_num += 1
            data, label = data.to(torch.float), label.to(torch.float)
            # 得到测试集的预测值
            y_pred = mlp(data)
            # 计算损失
            loss = test_criterion(y_pred, label)
            # 将损失值加入到本轮测试的损失中
            test_loss += loss.item()

    train_loss = 0
    # 在训练集中训练模型
    mlp.train()  # 关于。train() 的作用，可以参考 https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch
    train_batch_num = 0
    for data, label in tqdm(train_loader, f'Epoch {epoch}-train', leave=False):
        train_batch_num += 1
        # 准备数据
        data, label = data.to(torch.float), label.to(torch.float)
        # 得到训练集的预测值
        out_put = mlp(data)
        # 计算损失
        loss = criterion(out_put, label)
        # 将损失值加入到本轮训练的损失中
        train_loss += loss.item()
        # 梯度清零
        optimizer.zero_grad() # 关于。zero_grad() 的作用，可以参考 https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
        # 反向传播求解梯度
        loss.backward()
        # 更新权重参数
        optimizer.step()




    train_loss_list.append(train_loss/train_batch_num)
    test_loss_list.append(test_loss/test_batch_num)
print(train_loss_list)
print(test_loss_list)

